In [119]:
#1.Pulls details of banglore restaurants Zomato pagewise to and saves it to a csv.

In [62]:
import requests
from bs4 import BeautifulSoup
import re
import pandas
from collections import OrderedDict

In [63]:
startUrl = "https://www.zomato.com/bangalore/restaurants"

In [64]:
#Used headers/agent because the request was timed out and asking for an agent. 
#Using following code we can fake the agent.

def getSoup(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    response = requests.get(url,headers=headers)
    
    content = response.content
    soup = BeautifulSoup(content,"html.parser")
    print(url)
    return soup

In [65]:
#max pages
def getMaxPages():
    soup = getSoup(startUrl)
    pages = []
    pagesTag = soup.find("div",attrs={"class":"col-l-4 mtop pagination-number"})
    pagesTag_temp = pagesTag.find_all("b",recursive=True)
    
    for p in pagesTag_temp:
        pages.append(p.get_text())
        
    return 17#int(max(pages))


In [71]:

list_dataframes = []
currentpage = 16
maxPages = getMaxPages()
#print(maxPages)
while currentpage <= maxPages:
    
    pageUrl = startUrl+"?page="+str(currentpage)
    soup = getSoup(pageUrl)
    list_rest = soup.find_all("div",attrs={"class": "col-s-12"})
    
    for rest in list_rest:
        dataframe = OrderedDict() #dict

        #restaurant link
        link = rest.find("a",attrs={"class": "result-title hover_feedback zred bold ln24 fontsize0"})
        dataframe["restaurant_link"] = link['href']
        
        soup = getSoup(link['href'])
        
        #Res Id
        res_id = soup.find("head")
        scripts = res_id.select('script')
        for script in scripts:
            if 'window.RES_ID' in script.text:
                ids = re.findall('\d+', script.contents[0])
                dataframe["restaurant_ID"] = ids[0]
        
        #restaurant title
        top_header_title = (soup.find("h1",attrs={"class":"ui res-name mb0 header nowrap"}))
        dataframe["restaurant_name"] = top_header_title.find("a").get_text().strip()

        #locality
        loc_title = (soup.find("div",attrs={"class":"mb5 pt5 clear"}))
        dataframe["locality"] = loc_title.find("a").get_text().strip()

        #category
        res_category_tag = loc_title.find("span",attrs={"class":"res-info-estabs grey-text fontsize3"})
        if res_category_tag is not None:
            res_category = res_category_tag.find("a")
            if res_category is not None:
                dataframe["restaurant_category"] = res_category_tag.find("a").get_text().strip()
            else:
                dataframe["restaurant_category"] = res_category_tag.get_text().strip()#Delivery
        
        #zomato gold
        goldTag = soup.find("div",attrs={"class":"red_res_tag"})
        if goldTag is not None:
            goldTag_temp = goldTag.find("img")
            if goldTag_temp is not None:
                dataframe["zomato_gold"] = "zomato gold"
            else:
                dataframe["zomato_gold"] = "NA"
        else:
            dataframe["zomato_gold"] = "NA"

        #discounts
        discountTag = soup.find("div",attrs={"class": "fontsize flex flex-center-v"})
        if discountTag is not None:
            discountTag_temp = discountTag.find("div",attrs={"class":"pr5"})
            if discountTag_temp is not None:
                dataframe["discounts"] = discountTag_temp.getText().strip()
            else:
                dataframe["discounts"] = "NA"
        else:
            dataframe["discounts"] = "NA"
        
        #photos
        photosTag = soup.find_all("div",attrs={"respageMenuContainer"})
        for photo in photosTag:
            children = photo.findChildren("a",attrs={"item respageMenu-item photosTab restaurant-tab-item-jumbo-track"})
            for child in children:
                reObj = re.compile('\d+')
                dataframe["photos_taken"] = reObj.findall(child.text.strip())[0]

        #rating
        top_header_rating = (soup.find("div",attrs={"class":"col-l-4 tac left"}))
        rating = top_header_rating.find("div",attrs={"class":"rating_hover_popup res-rating pos-relative clearfix mb5"})

        if rating is not None:
            dataframe["rating"] = rating.get_text().replace('\n',' ').strip()
        else:
            dataframe["rating"] = "NA"

        #votes
        top_header_votes = top_header_rating.find("span",attrs={"class":"mt2 mb0 rating-votes-div rrw-votes grey-text fontsize5 ta-right"})
        if top_header_votes is not None:
            reObj = re.compile('\d+')
            dataframe["votes"] = reObj.findall(top_header_votes.get_text())[0]
        else:
            dataframe["votes"] = "NA"

        #cuisines
        info_left = (soup.find("div",attrs={"class":"col-l-1by3 pl0 pr20"}))
        cuisine_info = info_left.find("div",attrs={"class":"res-info-cuisines clearfix"})
        dataframe["cuisines"] = cuisine_info.get_text()

        #Approx cost
        approxCost_tag = info_left.find("div",attrs={"class":"res-info-detail"})
        if approxCost_tag is not None:
            approx_cost = approxCost_tag.find("span",attrs={"tabindex":"0"})
        if approx_cost is not None:
            reObj = re.compile('\d+')
            dataframe["approx_cost_for_2"] = reObj.findall(approx_cost.get_text().replace(',','').strip())[0]
        else:
            dataframe["approx_cost_for_2"] = "NA"
        
        #Opening timings
        info_centre = (soup.find("div",attrs={"class":"col-l-1by3 pl20 pr20"}))
        if info_centre is not None:
            openingtimes = info_centre.find("div",attrs={"class":"res-info-timings"})
            if openingtimes is not None:
                timings = openingtimes.find("div",attrs={"class":"medium"})
                dataframe["opening_timings"] = re.findall(r'\s(\d*\:\d*\s?(?:AM|PM|am|pm|noon|midnight|Hours|hours)|\d*\s?(?:AM|PM|am|pm|noon|midnight|Hours|hours))',timings.get_text())
            else:
                dataframe["opening_timings"] = "NA"
        
        #address
        addressTag = info_centre.find("div",attrs={"class":"mbot0"})
        address = addressTag.find("span")
        if addressTag and address is not None:
            dataframe["address"] = address.get_text()
        else:
            dataframe["address"] = "NA"
        
        #lat-long
        lat_longTag = soup.find("div",attrs={"class":"resmap pos-relative mt5 mb5"})
        if lat_longTag is not None:
            lat_long_scripts = lat_longTag.select('script')
            for script in lat_long_scripts:
                reObj = re.compile('\d*\.\d*')
                lat_lon = reObj.findall(script.contents[0])
                
                if lat_lon is not None:
                    dataframe["latitude"] = lat_lon[0]
                    dataframe["longitude"] = lat_lon[1]
                else:
                    dataframe["longitude"] = dataframe["latitude"] = "NA"

        #more info
        more_info = (soup.find_all("div",attrs={"class":"res-info-highlights"}))
        if more_info == []:
            dataframe["more_info"] = "NA"
        else:
            for child in more_info:
                infos = child.find_all("div",attrs={"class":"res-info-feature-text"})
                infostring = []
                for info in infos:
                    infostring.append(info.get_text())
                dataframe["more_info"] = infostring
            
        #Featured in Collections
        featured_in_tag = soup.find("div",attrs={"class":"mbot0 ptop0"})
        if featured_in_tag is not None:
            featured_in = featured_in_tag.find("div",attrs={"class":"resbox__main--row res-info-group clearfix"})

            if featured_in is not None:
                features = featured_in.find_all("a")
            if features is not None: 
                dataframe["featured_in"] = [ftr.get_text().strip() for ftr in features]
            else:
                dataframe["featured_in"] = "NA"
        else:
            dataframe["featured_in"] = "NA"
            
            
        #known for
        knownForTag = soup.find_all("div",attrs={"class":"mbot0 ptop0"})
        if knownForTag == []:
            dataframe["known_for"] = "NA"
        else:
            for child in knownForTag:
                known_for_list = child.find_all("div",attrs={"class":"res-info-known-for-text mr5"})
                for known_for in known_for_list:
                    dataframe["known_for"] = known_for.get_text().strip()            

        #loved here food
        mostlovedTags = soup.find_all("div",attrs={"class":"rv_highlights__wrapper mtop0"})
        j = []
        for child in mostlovedTags:
            j = child.find_all("div",attrs={"class":"rv_highlights__section pr10"})
        
        if j == []:
            dataframe["most_liked_Food"] = "NA"
            dataframe["most_liked_Service"] = "NA"
            dataframe["most_liked_Look & Feel"] = "NA"
        else:    
            for op in j:
                mostlovedcategory = op.find("div",attrs={"class":"grey-text"})

                loved_here = op.find("div",attrs={"class":"rv_highlights__score_bar mt5 mb5"})
                blockbg = loved_here.find_all("div",attrs={"class":"block bd-txt-bg"})

                mostLoved = op.find("div",attrs={"class":"fontsize13 ln18"})
                if mostLoved is not None:
                    item = mostLoved.find_all("span")
                    if item is not None:
                        temp = []
                        temp.append(str(5-len(blockbg))+"/5")
                        for i in item:
                            temp.append(i.get_text().replace(',','').strip())
                        dataframe["most_liked_"+mostlovedcategory.get_text()] = temp 

        ##reviews
        review_link = link['href']+"/reviews"
        review_soup = getSoup(review_link)

        review_header = (review_soup.find_all("div",attrs={"class":"res-reviews-container res-reviews-area"}))
        review_list = []
        for i in review_header:
            main = []
            rev_details_parent = i.findChildren("div",attrs={'fs12px pbot0 clearfix'})
            for j in rev_details_parent:
                rev_info = []
                rev_info.append(j.time['datetime'])
                if j.find("div",attrs={"class":"positive-sentiment"}):
                    rev_info.append("positive")
                else:
                    rev_info.append("NA")
                    
                if j.find("div",attrs={"class":"negative-sentiment"}):
                    rev_info.append("negative")
                else:
                    rev_info.append("NA")
                    
                main.append(rev_info)
            
            rev_children = i.findChildren("div",attrs={"rev-text mbot0","rev-text-sm mbot0"})
            
            for indx,child in enumerate(rev_children):
                reviewtxt = child.text.replace('Rated\xa0\n','').strip()
                main[indx].append(reviewtxt.replace('\n',''))
                
        dataframe["reviews"] = tuple(main)
        #print(dataframe["reviews"])
        list_dataframes.append(dataframe)

    currentpage += 1
else:
    currentpage = 1
        



https://www.zomato.com/bangalore/restaurants
https://www.zomato.com/bangalore/restaurants?page=16
https://www.zomato.com/bangalore/21st-amendment-gastrobar-indiranagar-bangalore
https://www.zomato.com/bangalore/21st-amendment-gastrobar-indiranagar-bangalore/reviews
https://www.zomato.com/bangalore/vanilla-sky-hsr-bangalore
https://www.zomato.com/bangalore/vanilla-sky-hsr-bangalore/reviews
https://www.zomato.com/bangalore/le-grapes-restobar-rajajinagar-bangalore
https://www.zomato.com/bangalore/le-grapes-restobar-rajajinagar-bangalore/reviews
https://www.zomato.com/bangalore/om-made-cafe-koramangala-5th-block
https://www.zomato.com/bangalore/om-made-cafe-koramangala-5th-block/reviews
https://www.zomato.com/bangalore/sharief-bhai-1-koramangala-5th-block
https://www.zomato.com/bangalore/sharief-bhai-1-koramangala-5th-block/reviews
https://www.zomato.com/bangalore/ji-hazoor-whitefield
15% off on your first order. Only applicable on orders above ₹500.
https://www.zomato.com/bangalore/ji-haz

In [72]:
df = pandas.DataFrame(list_dataframes,columns=list_dataframes[0].keys())
df.to_csv("zomato_extracted_data_csv/zomato_extracted_data"+".csv",index=False)
print(list_dataframes[0].keys())

odict_keys(['restaurant_link', 'restaurant_ID', 'restaurant_name', 'locality', 'restaurant_category', 'zomato_gold', 'discounts', 'photos_taken', 'rating', 'votes', 'cuisines', 'approx_cost_for_2', 'opening_timings', 'address', 'latitude', 'longitude', 'more_info', 'featured_in', 'most_liked_Food', 'most_liked_Service', 'most_liked_Look & Feel', 'reviews'])
